<a href="https://colab.research.google.com/github/mgo-city/INM707-lab06/blob/main/notenooks/INM707_Lab06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gymnasium

In [ ]:
pip install -U "ray[rllib]"

In [ ]:
!pip install git+https://github.com/mgo-city/INM707-lab06.git

In [6]:
from dungeon.dungeon import Dungeon, index_to_actions

In [7]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np

In [8]:
class DungeonEnv(gym.Env):

    def __init__(self, render_mode=None, size=5):
        self.size = size  # The size of the Dungeon

        # Observations are dictionaries with the agent's and the target's location.
        # Each location is encoded as an element of {0, ..., `size`}^2, i.e. MultiDiscrete([size, size]).
        self.observation_space = spaces.Dict(
            {
                "relative_coordinates": spaces.Box(-size, size , shape=(2,), dtype=int),
                "surroundings": spaces.Box(0, 4, shape=(5, 5), dtype=int)
            }
        )

        # We have 4 actions, corresponding to "right", "up", "left", "down"
        self.action_space = spaces.Discrete(4)

        self.render_mode = "ansi"

        self.clock = None

        self._dungeon_env = Dungeon(size)
        self._dungeon_env.reset()


    def reset(self, seed=None, options=None):
        # We need the following line to seed self.np_random
        super().reset(seed=seed)

        obs = self._dungeon_env.reset()

        return obs, {}

    def step(self, action):

        act = index_to_actions[action]

        observations, reward, done = self._dungeon_env.step(act.name)

        return observations, reward, done, False, {}

    def render(self):

        envir_with_agent = self._dungeon_env.dungeon.copy()
        envir_with_agent[self._dungeon_env.position_agent[0], self._dungeon_env.position_agent[1]] = 4
        
        full_repr = ""

        for r in range(self.size):
            
            line = ""
            
            for c in range(self.size):

                string_repr = self._dungeon_env.dict_map_display[ envir_with_agent[r,c] ]
                
                line += "{0:2}".format(string_repr)

            full_repr += line + "\n"

        return full_repr


In [9]:
dungeon_env = DungeonEnv(size=100)


In [10]:
dungeon_env.render()
dungeon_env.step(1)

({'relative_coordinates': array([80,  8]),
  'surroundings': array([[0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]], dtype=int8)},
 -1,
 False,
 False,
 {})

In [ ]:
import ray
from ray.rllib.algorithms import ppo, dqn
from ray.tune.logger import pretty_print

In [ ]:
ray.shutdown()
ray.init()

algo = dqn.DQN(env=DungeonEnv, config={
    "env_config": {"size":40},  # config to pass to env class
})

while True:
    metrics = algo.train()
    mean_reward = metrics['episode_reward_mean']
    print(mean_reward)


2023-03-29 09:49:54,517	INFO worker.py:1553 -- Started a local Ray instance.
2023-03-29 09:49:55,875	WARNING deprecation.py:50 -- DeprecationWarning: `algo = Algorithm(env='<class '__main__.DungeonEnv'>', ...)` has been deprecated. Use `algo = AlgorithmConfig().environment('<class '__main__.DungeonEnv'>').build()` instead. This will raise an error in the future!
2023-03-29 09:49:55,882	INFO algorithm_config.py:2899 -- Your framework setting is 'tf', meaning you are using static-graph mode. Set framework='tf2' to enable eager execution with tf2.x. You may also then want to set eager_tracing=True in order to reach similar execution speed as with static-graph mode.
2023-03-29 09:49:55,916	INFO algorithm.py:506 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
2023-03-29 09:49:55,930	WARNING env.py:156 -- Your env doesn't have a .spec.max_episode_steps attribute. Your horizon will default to infinity, and your environment wil

-129.22727272727272
-90.43
-53.1588785046729
